In [1]:
from pymatsolver import Pardiso
import numpy as np
import scipy as sp
import dask
import dask.array as da
from pyMKL import mkl_set_num_threads

import time


In [19]:
# Create a simple sparse banded system
nC = 100000
A = sp.sparse.diags([-np.ones(nC), 2*np.ones(nC), -np.ones(nC)], [-1,0,1], shape=(nC,nC))

num_threads = 8
mkl_set_num_threads(num_threads)
# Create Pardiso solver for Ainv
Ainv = Pardiso(A)

# Solve it for a random RHS
b = np.random.randn(nC,1000)



In [20]:

# Solve the RHS in a loop using pymatsolver
y = Ainv*b # Solve once to get the factors

start = time.time()
y = Ainv*b
print("Solve in: {}".format(time.time()-start))
y[:, 0]

Solve in: 9.808005809783936


array([-120.58956495, -240.58346634, -360.30479917, ..., -849.30553014,
       -566.28661642, -283.98403352])

In [21]:
# Try to do the same solves inside a delayed function
@dask.delayed
def solveRHD(rhs):

    return Ainv*rhs

# Split the solves in dask delays
columns = [solveRHD(b[:, ii]).reshape((nC,1)) for ii in range(b.shape[1])]
solves = [da.from_delayed(column, dtype='float', shape=(nC, 1)).reshape((nC,1)) for column in columns]

# Stack the result
yDask = da.hstack(solves)

In [22]:
# Compute only one of them 
# First is the same ... good
solves[0].compute()




array([[-120.58956495],
       [-240.58346634],
       [-360.30479917],
       ...,
       [-849.30553014],
       [-566.28661642],
       [-283.98403352]])

In [26]:
# Run all with dask ... not good, run again different
num_threads = 1
mkl_set_num_threads(num_threads)
start = time.time()
yOut = yDask.compute()
print("Solve in: {}".format(time.time()-start))
yOut[:,0]

Solve in: 6.7459962368011475


array([ -36.06651756,  -72.73359207, -106.27685574, ..., -267.0024918 ,
       -178.48359654,  -89.28545671])

In [15]:
6/9.45



0.634920634920635